In [1]:
import sys
import os

# Add the directory containing your modules to the system path
sys.path.insert(0, os.path.abspath('/projects/e32704/andy/decisionmaking/gridworld-decisionmaking/Simulation 2 Pseudo-Terrestrial/planning'))

In [2]:
import pickle as pkl
import pandas as pd
import numpy as np

In [3]:
from game import Game
from simulator import Knowledge
from mcts import SearchParams, UnitTestMCTS
from experiment import ExperimentParams, ExperimentParamsOld, Experiment
from entropy import UnitTestENTROPY
from utils import UnitTestUTILS, Random
from coord import COORD, UnitTestCOORD, COMPASS, ManhattanDistance, Compass, LineCoordinates
from grid import Grid
from entropy import Entropy
from astar import AStar
from math import floor
from time import time

In [4]:
def GetPath(occlusions):
    ASTAR = AStar(XSize, YSize, occlusions)
    ASTAR.InitializeGrid(AgentHome, GoalPos)
    path = ASTAR.Solve()

    return path

def GetEntropy(occlusions):
    I = np.zeros((XSize, YSize))
    for coord in occlusions:
        I[coord.X, coord.Y] = 1
    entropy = Entropy(I)
    outputMatrix = entropy.MovingWindowFilter(entropy.MovingAverage, 1)
    filteredMatrices = [outputMatrix]
    profile = entropy.Profile(filteredMatrices)

    return profile


## 3/17 Examinations

```python
class ExperimentParamsOld:
    SpawnArea = 4
    NumRuns = 20
    NumPredators = 5
    NumSteps = 200
    SimSteps = 1000
    TimeOut = 36000
    MinDoubles = 0
    MaxDoubles = 50
    TransformDoubles = -1
    TransformAttempts = 1000
    Accuracy = 0.01
    UndiscountedHorizon = 100
    AutoExploration = True
    EntropyLevels = [float(i)/10. for i in range(0, 10)]
    Depth = [100, 1000, 5000]
```

In [5]:
XSize = 15
YSize = 15
AgentHome = COORD(7, 0)
GoalPos = COORD(floor(XSize/2), YSize-1)
visualRange = None


treeknowlege = 2 # 0 = pure, 1 = legal, 2 = smart
rolloutknowledge = 2 # 0 = pure, 1 = legal, 2 = smart
smarttreecount = 1.0 # prior count for preferred actions in smart tree search
smarttreevalue = 1.0 # prior value for preferred actions during smart tree search

In [17]:
def get_init_vars(experiment_params):
    occlusionList = [[[] for occlusionInd in experiment_params.EntropyLevels]
                        for simulationInd in range(experiment_params.NumRuns)]
    predatorList = [[[] for occlusionInd in experiment_params.EntropyLevels]
                     for simulationInd in range(experiment_params.NumRuns)]
    for simulationInd in range(experiment_params.NumRuns):
        for i, entropyValue in enumerate(experiment_params.EntropyLevels):
            while True and entropyValue < .8:
                numOcclusions = 0
                occlusions = Grid(XSize, YSize).CreateOcclusions(numOcclusions, AgentHome, GoalPos)
                while GetEntropy(occlusions)[0] < entropyValue:
                    #entropyMatrix = GetEntropy(occlusions)[0]
                    #if entropyMatrix >= entropyValue:
                    #    break
                    occlusions = Grid(XSize, YSize).CreateOcclusions(numOcclusions, AgentHome, GoalPos)
                    numOcclusions += 1
                path = GetPath(occlusions)
                if path:
                    occlusionList[simulationInd][i] = occlusions
                    break
    
            while True and entropyValue >= .8:
                numOcclusions = 20
                occlusions = Grid(XSize, YSize).CreateRandomOcclusions(numOcclusions, AgentHome, GoalPos)
                while GetEntropy(occlusions)[0] < entropyValue:
                    occlusions = Grid(XSize, YSize).CreateOcclusions(numOcclusions, AgentHome, GoalPos)
                    numOcclusions += 1
    
                path = GetPath(occlusions)
                if path:
                    occlusionList[simulationInd][i] = occlusions
                    break
    
            allPredatorLocations = Grid(XSize, YSize).CreatePredatorLocations(experiment_params.SpawnArea, AgentHome,
                                                                      GoalPos,occlusions)
    
            temp_predator = [allPredatorLocations[Random(0, len(allPredatorLocations))]
                                 for p in range(experiment_params.NumPredators)]
            predatorList[simulationInd][i] = temp_predator

    return [occlusionList, predatorList]

def get_tasks(experiment_params, occlusionList, predatorList, visualRange=None, planningDir='/testpath'):
    tasks = []
    for simulationInd in range(experiment_params.NumRuns):
        for occlusionInd in range(len(experiment_params.EntropyLevels)):
            for predatorInd in range(experiment_params.NumPredators):
                if visualRange:
                    for visrange in visualRange:
                        tasks.append([(occlusionInd, occlusionList[simulationInd][occlusionInd]),
                                      (predatorInd, predatorList[simulationInd][occlusionInd][predatorInd]), visrange,
                                      simulationInd, planningDir])
                tasks.append([(occlusionInd, occlusionList[simulationInd][occlusionInd]),
                        (predatorInd, predatorList[simulationInd][occlusionInd][predatorInd]), None, simulationInd, planningDir])
    return tasks

In [7]:
exp_params = ExperimentParams()
exp_params_old = ExperimentParamsOld()

In [8]:
print(exp_params_old)

ExperimentParams:
	SpawnArea: 4
	NumRuns: 20
	NumPredators: 5
	NumSteps: 200
	SimSteps: 1000
	TimeOut: 36000
	MinDoubles: 0
	MaxDoubles: 50
	TransformDoubles: -1
	TransformAttempts: 1000
	Accuracy: 0.01
	UndiscountedHorizon: 100
	AutoExploration: True
	EntropyLevels: [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
	Depth: [100, 1000, 5000]



In [9]:
init_vars = get_init_vars(exp_params)
init_vars_old = get_init_vars(exp_params_old)

In [46]:
tasks[0]

[(0, []), (0, <coord.COORD at 0x7f81964cb150>), None, 0, '/testpath']

In [47]:
tasks[2]

[(2,
   <coord.COORD at 0x7f81964c9310>]),
 (0, <coord.COORD at 0x7f81964c8dd0>),
 None,
 0,
 '/testpath']

In [48]:
coords_lst = tasks[2][0][1]

In [49]:
coords_lst

In [50]:
for coord in coords_lst:
    print(coord)

(13;0)
(14;0)
(13;4)
(13;5)
(14;5)
(14;6)
(14;7)
(10;9)


In [51]:
test_coord = COORD(14, 7)

In [52]:
print(test_coord)

(14;7)


In [53]:
test_coord in coords_lst

True

In [20]:
tasks = get_tasks(exp_params, *init_vars_old)
tasks_old = get_tasks(exp_params_old, *init_vars_old)

In [22]:
len(tasks)

4

In [23]:
len(tasks_old)

1000

In [25]:
set(len(task) for task in tasks)

{5}

In [26]:
set(len(task) for task in tasks_old)

{5}

In [33]:
currentTask = tasks_old[0]

In [35]:
currentTask

[(0, []), (0, <coord.COORD at 0x7f81964cb150>), None, 0, '/testpath']

In [34]:
print(" Starting Simulation %d, Entropy 0.%d, Predator %d, Visual Range %s"
                            %(currentTask[3], currentTask[0][0], currentTask[1][0], str(currentTask[2])))

 Starting Simulation 0, Entropy 0.0, Predator 0, Visual Range None


In [36]:
def print_curr_task(currentTask):
    print(" Starting Simulation %d, Entropy 0.%d, Predator %d, Visual Range %s"
                            %(currentTask[3], currentTask[0][0], currentTask[1][0], str(currentTask[2])))

In [38]:
ct = tasks_old[890]

In [41]:
ct[0][0]

8

In [37]:
print_curr_task(tasks_old[890])

 Starting Simulation 17, Entropy 0.8, Predator 0, Visual Range None


In [32]:
for task in tasks_old[890:895]:
    print(task)
    print()

[(8, [<coord.COORD object at 0x7f81965617d0>, <coord.COORD object at 0x7f819655ba90>, <coord.COORD object at 0x7f819655b6d0>, <coord.COORD object at 0x7f8196559b10>, <coord.COORD object at 0x7f8196558b10>, <coord.COORD object at 0x7f8196559790>, <coord.COORD object at 0x7f8196558c50>, <coord.COORD object at 0x7f81965594d0>, <coord.COORD object at 0x7f819655a6d0>, <coord.COORD object at 0x7f819655a510>, <coord.COORD object at 0x7f8196559f50>, <coord.COORD object at 0x7f819655b090>, <coord.COORD object at 0x7f8196559950>, <coord.COORD object at 0x7f819655a410>, <coord.COORD object at 0x7f8196559150>, <coord.COORD object at 0x7f819655ab90>, <coord.COORD object at 0x7f8196559290>, <coord.COORD object at 0x7f819655a1d0>, <coord.COORD object at 0x7f81965591d0>, <coord.COORD object at 0x7f819655bc10>, <coord.COORD object at 0x7f8196558250>, <coord.COORD object at 0x7f8196568350>, <coord.COORD object at 0x7f8196568550>, <coord.COORD object at 0x7f819656aed0>, <coord.COORD object at 0x7f8196568

In [11]:
occlusionList, predatorList = init_vars_old
for simulationInd in range(exp_params_old.NumRuns):
    print(f"Simulation {simulationInd}:")
    for occlusionInd, entropyLevel in enumerate(exp_params_old.EntropyLevels):
        print(f"\tEntropy Level {entropyLevel}:")
        print(f"\t\tPredator Locations: {[str(loc) for loc in predatorList[simulationInd][occlusionInd]]}")

Simulation 0:
	Entropy Level 0.0:
		Predator Locations: ['(1;7)', '(3;5)', '(12;2)', '(1;4)', '(12;7)']
	Entropy Level 0.1:
		Predator Locations: ['(3;6)', '(2;12)', '(2;10)', '(10;8)', '(0;5)']
	Entropy Level 0.2:
		Predator Locations: ['(13;1)', '(12;10)', '(11;11)', '(0;6)', '(13;6)']
	Entropy Level 0.3:
		Predator Locations: ['(11;14)', '(14;6)', '(11;10)', '(10;7)', '(11;6)']
	Entropy Level 0.4:
		Predator Locations: ['(12;12)', '(7;12)', '(6;8)', '(1;1)', '(11;6)']
	Entropy Level 0.5:
		Predator Locations: ['(0;12)', '(13;6)', '(14;5)', '(11;10)', '(2;14)']
	Entropy Level 0.6:
		Predator Locations: ['(6;12)', '(6;8)', '(5;8)', '(0;11)', '(4;8)']
	Entropy Level 0.7:
		Predator Locations: ['(3;9)', '(13;4)', '(9;9)', '(9;8)', '(1;2)']
	Entropy Level 0.8:
		Predator Locations: ['(12;2)', '(2;0)', '(12;4)', '(8;7)', '(1;9)']
	Entropy Level 0.9:
		Predator Locations: ['(14;14)', '(13;14)', '(13;8)', '(1;10)', '(7;13)']


## Old Examinations

In [3]:
init_vars_path = "/projects/e32704/andy/decisionmaking/gridworld-results/initial_test/init_vars.pkl"

In [4]:
with open(init_vars_path, 'rb') as f:
    init_vars = pkl.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/projects/e32704/andy/decisionmaking/gridworld-results/initial_test/init_vars.pkl'

In [9]:
type(init_vars)

list

In [10]:
len(init_vars)

2

In [12]:
type(init_vars[0])

list

In [13]:
type(init_vars[1])

list

In [14]:
len(init_vars[0])

20

In [15]:
len(init_vars[1])

20

In [17]:
type(init_vars[0][0])

list

In [19]:
len(init_vars[0][0])

10

In [20]:
init_vars[0][0][0]

[]

In [21]:
init_vars[0][0][1]

In [22]:
init_vars[0][0][2]

In [28]:
for i, val in enumerate(init_vars[0][0]):
    # print(type(val))
    # print(len(val))
    if len(val) > 0:
        # print(val[1])
        print(len(val) == len(set(val)))

True
False
False
False
False
False
False
False
False


My guess is that `init_vars[0][0]` specifies the occlusions for an environment. Specifically, I believe that the second index specifies occlusions - the first index is likely some other high-level factor, since there are only two items at the first index.

### Step through construction logic

In [6]:
XSize = 15
YSize = 15
AgentHome = COORD(7, 0)
GoalPos = COORD(floor(XSize/2), YSize-1)
visualRange = None

In [58]:
def VisualCone(self, coord, visualRange):
    temp = []
    for dec in range(visualRange):
        temp.append(coord[dec, dec:(2 * visualRange - dec) + 1])
    return [y for x in temp for y in x]  # if self.Inside(y)]

In [65]:
def VisualArea(self, coord, observationDirection, visualRange, pureVision=False):
    RadiusCoordinates = []
    for x in range(coord.X - visualRange, coord.X + visualRange + 1):
        for y in range(coord.Y - visualRange, coord.Y + visualRange + 1):
            if (ManhattanDistance(COORD(x, y), coord) <= 2 * visualRange):
                RadiusCoordinates.append(COORD(x, y))

    RangeCoordinates = []
    RadiusCoordinates = np.flipud(
        np.reshape(np.array(RadiusCoordinates), (2 * visualRange + 1, 2 * visualRange + 1)).transpose())
    if observationDirection == COMPASS.NORTH:
        RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

    elif observationDirection == COMPASS.EAST:
        RadiusCoordinates = np.flipud(RadiusCoordinates.transpose())
        RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

    elif observationDirection == COMPASS.WEST:
        RadiusCoordinates = RadiusCoordinates.transpose()
        RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

    elif observationDirection == COMPASS.SOUTH:
        RadiusCoordinates = np.flipud(RadiusCoordinates)
        RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

    assert (RangeCoordinates)

    if pureVision:
        return RangeCoordinates

    for a in range(4):
        sidePos = coord + Compass[a]
        if sidePos not in RangeCoordinates:
            RangeCoordinates.append(sidePos)

    return RangeCoordinates

In [79]:
observationDirection = COMPASS.NORTH
RadiusCoordinates = []
for x in range(coord.X - visualRange, coord.X + visualRange + 1):
    for y in range(coord.Y - visualRange, coord.Y + visualRange + 1):
        print(x, y)
        if (ManhattanDistance(COORD(x, y), coord) <= 2 * visualRange):
            RadiusCoordinates.append(COORD(x, y))

# RangeCoordinates = []
# RadiusCoordinates = np.flipud(
#     np.reshape(np.array(RadiusCoordinates), (2 * visualRange + 1, 2 * visualRange + 1)).transpose())
# if observationDirection == COMPASS.NORTH:
#     RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

# elif observationDirection == COMPASS.EAST:
#     RadiusCoordinates = np.flipud(RadiusCoordinates.transpose())
#     RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

# elif observationDirection == COMPASS.WEST:
#     RadiusCoordinates = RadiusCoordinates.transpose()
#     RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

# elif observationDirection == COMPASS.SOUTH:
#     RadiusCoordinates = np.flipud(RadiusCoordinates)
#     RangeCoordinates = self.VisualCone(RadiusCoordinates, visualRange)

# assert (RangeCoordinates)

# if pureVision:
#     return RangeCoordinates

# for a in range(4):
#     sidePos = coord + Compass[a]
#     if sidePos not in RangeCoordinates:
#         RangeCoordinates.append(sidePos)

# return RangeCoordinates

2 -5
2 -4
2 -3
2 -2
2 -1
2 0
2 1
2 2
2 3
2 4
2 5
3 -5
3 -4
3 -3
3 -2
3 -1
3 0
3 1
3 2
3 3
3 4
3 5
4 -5
4 -4
4 -3
4 -2
4 -1
4 0
4 1
4 2
4 3
4 4
4 5
5 -5
5 -4
5 -3
5 -2
5 -1
5 0
5 1
5 2
5 3
5 4
5 5
6 -5
6 -4
6 -3
6 -2
6 -1
6 0
6 1
6 2
6 3
6 4
6 5
7 -5
7 -4
7 -3
7 -2
7 -1
7 0
7 1
7 2
7 3
7 4
7 5
8 -5
8 -4
8 -3
8 -2
8 -1
8 0
8 1
8 2
8 3
8 4
8 5
9 -5
9 -4
9 -3
9 -2
9 -1
9 0
9 1
9 2
9 3
9 4
9 5
10 -5
10 -4
10 -3
10 -2
10 -1
10 0
10 1
10 2
10 3
10 4
10 5
11 -5
11 -4
11 -3
11 -2
11 -1
11 0
11 1
11 2
11 3
11 4
11 5
12 -5
12 -4
12 -3
12 -2
12 -1
12 0
12 1
12 2
12 3
12 4
12 5


In [60]:
visualRange = 5

In [62]:
coord = AgentHome

In [63]:
temp = []
for dec in range(visualRange):
    temp.append(coord[dec, dec:(2 * visualRange - dec) + 1])
return [y for x in temp for y in x]  # if self.Inside(y)]

TypeError: 'COORD' object is not subscriptable

In [43]:
occlusions = Grid(XSize, YSize).CreateOcclusions(numOcclusions, AgentHome, GoalPos)

In [7]:
def GetEntropy(occlusions):
    I = np.zeros((XSize, YSize))
    for coord in occlusions:
        I[coord.X, coord.Y] = 1
    entropy = Entropy(I)
    outputMatrix = entropy.MovingWindowFilter(entropy.MovingAverage, 1)
    filteredMatrices = [outputMatrix]
    profile = entropy.Profile(filteredMatrices)

    return profile

In [11]:
def GetPath(occlusions):
    ASTAR = AStar(XSize, YSize, occlusions)
    ASTAR.InitializeGrid(AgentHome, GoalPos)
    path = ASTAR.Solve()

    return path

In [12]:
occlusionList = [[[] for occlusionInd in ExperimentParams.EntropyLevels]
                        for simulationInd in range(ExperimentParams.NumRuns)]
predatorList = [[[] for occlusionInd in ExperimentParams.EntropyLevels]
                 for simulationInd in range(ExperimentParams.NumRuns)]
for simulationInd in range(ExperimentParams.NumRuns):
    for i, entropyValue in enumerate(ExperimentParams.EntropyLevels):
        while True and entropyValue < .8:
            numOcclusions = 0
            occlusions = Grid(XSize, YSize).CreateOcclusions(numOcclusions, AgentHome, GoalPos)
            while GetEntropy(occlusions)[0] < entropyValue:
                #entropyMatrix = GetEntropy(occlusions)[0]
                #if entropyMatrix >= entropyValue:
                #    break
                occlusions = Grid(XSize, YSize).CreateOcclusions(numOcclusions, AgentHome, GoalPos)
                numOcclusions += 1
            path = GetPath(occlusions)
            if path:
                occlusionList[simulationInd][i] = occlusions
                break

        while True and entropyValue >= .8:
            numOcclusions = 20
            occlusions = Grid(XSize, YSize).CreateRandomOcclusions(numOcclusions, AgentHome, GoalPos)
            while GetEntropy(occlusions)[0] < entropyValue:
                occlusions = Grid(XSize, YSize).CreateOcclusions(numOcclusions, AgentHome, GoalPos)
                numOcclusions += 1

            path = GetPath(occlusions)
            if path:
                occlusionList[simulationInd][i] = occlusions
                break

        allPredatorLocations = Grid(XSize, YSize).CreatePredatorLocations(ExperimentParams.SpawnArea, AgentHome,
                                                                  GoalPos,occlusions)

        temp_predator = [allPredatorLocations[Random(0, len(allPredatorLocations))]
                             for p in range(ExperimentParams.NumPredators)]
        predatorList[simulationInd][i] = temp_predator

### Notes

Each of `occlusionList` and `predatorList` is a list, and specifically a 2d list where the outer list represents the number of simulation runs (`ExperimentParams.NumRuns`, 20 runs) and the inner list represents different entropy levels (`ExperimentParams.EntropyLevels`, 10 levels).

A unique set of occlusions is generated for 



* index 0 corresponds with two lists - 0 is the occlusionList, and 1 is the predatorList
* 